In [1]:
!pip install -q /kaggle/input/language-tool-python-2-7-1/language_tool_python-2.7.1-py3-none-any.whl

In [2]:
import os
import zipfile
from zipfile import ZipFile
import shutil

# create download path
def get_language_tool_cache_path():
    # Get download path from environment or use default.
    download_path = os.environ.get(
        'LTP_PATH',
        os.path.join(os.path.expanduser("~"), ".cache", "language_tool_python")
    )
    # Make download path, if it doesn't exist.
    os.makedirs(download_path, exist_ok=True)
    return download_path

lt_path = get_language_tool_cache_path()

def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths        
  
def main():
    # path to folder which needs to be zipped
    directory = '../input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'
  
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)

    # writing files to a zipfile
    with ZipFile('./lt.zip','w') as zip:
        # writing each file one by one
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
    
main()

zip_file = "./lt.zip"
 
try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall()
        print("Extracted all")
except:
    print("Invalid file")
    
#move to cache
!mv {'./input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'} {lt_path} 
print(os.listdir('/root/.cache/language_tool_python/'))

#remove files from output

shutil.rmtree('./input')
os.remove("./lt.zip")

All files zipped successfully!
Extracted all
['LanguageTool-5.7']


In [3]:
import pandas as pd
from arrow import now
from language_tool_python import LanguageTool

time_start = now()
corrector = LanguageTool('en-US')


def score(arg: str, ) -> float:
    result = 0
#     result += 0.3 if len(arg) < 1798 else 0
#     result += 0.3 if len(arg.split()) < 290 else 0
    result += 1 if arg == corrector.correct(arg) else 0
    return result

print('built scoring function in {}'.format(now() - time_start))

built scoring function in 0:00:01.683384


In [4]:
time_start = now()
test_df = pd.read_csv(filepath_or_buffer='../input/llm-detect-ai-generated-text/test_essays.csv')
test_df['generated'] = test_df['text'].apply(func=score, )
print('generated scores in {}'.format(now() - time_start))

generated scores in 0:00:05.125127


In [5]:
test_df.drop(columns=['prompt_id', 'text']).to_csv(path_or_buf='submission.csv', index=False)